In [57]:
%load_ext autoreload
%autoreload 2

import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nwsrfs_models.main as s
from utilities.model import Model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
s.__name__

In [10]:
folder='C:\\Users\\geoffrey.walters\\Documents\\AORC\\AutoCalb\\Repo\\rfc-hydro-models\\py-rfchydromodels\\working'

In [6]:
ts_test=pd.read_csv(os.path.join(folder,'Flow_Test.csv'),skiprows=1,index_col='Hour')
ts=ts_test.Inflow.astype('f4').to_numpy()

conw1_chps=pd.read_excel(os.path.join(folder,'CONW1R_Lagk_Check.xlsx'),'Flows',index_col='Date_Time',parse_dates=True)
conw1=conw1_chps.CONW1.astype('f4').to_numpy()

sakw1_chps=pd.read_excel(os.path.join(folder,'SAKW1R_Lagk_Check.xlsx'),'Flows',index_col='Date_Time',parse_dates=True)
sakw1=sakw1_chps.SAKW1.astype('f4').to_numpy()

In [7]:
pars_1zone

,p_name,name,type,zone,value
0,adc_a_TLMO3-1,adc_a,snow,TLMO3-1,0.730028
1,adc_a_TLMO3-2,adc_a,snow,TLMO3-2,0.373551
2,adc_b_TLMO3-1,adc_b,snow,TLMO3-1,9.435536
3,adc_b_TLMO3-2,adc_b,snow,TLMO3-2,8.302735
4,adc_c_TLMO3-1,adc_c,snow,TLMO3-1,1.957479
...,...,...,...,...,...
313,uztwm_TLMO3-2,uztwm,sac,TLMO3-2,125.351540
314,zone_area_TLMO3-1,zone_area,sac,TLMO3-1,275.349000
315,zone_area_TLMO3-2,zone_area,sac,TLMO3-2,143.895000
316,zperc_TLMO3-1,zperc,sac,TLMO3-1,80.186588


In [71]:
#pd.read_csv(os.path.join(folder,'basins-1zone','TLMO3','results_01','pars_optimal.csv'))
pd.read_csv(os.path.join(folder,'basins_1zone','pars_optimal.csv'))

,p_name,name,type,zone,value
0,adc_a_TLMO3-1,adc_a,snow,TLMO3-1,0.498158
1,adc_b_TLMO3-1,adc_b,snow,TLMO3-1,0.679327
2,adc_c_TLMO3-1,adc_c,snow,TLMO3-1,8.352456
3,adimp_TLMO3-1,adimp,sac,TLMO3-1,0.095745
4,alat_TLMO3-1,alat,snow,TLMO3-1,45.578000
...,...,...,...,...,...
162,uzfwm_TLMO3-1,uzfwm,sac,TLMO3-1,83.566467
163,uzk_TLMO3-1,uzk,sac,TLMO3-1,0.159197
164,uztwm_TLMO3-1,uztwm,sac,TLMO3-1,284.494580
165,zone_area_TLMO3-1,zone_area,sac,TLMO3-1,417.566000


In [113]:
pars_1zone = pd.read_csv(os.path.join(folder,'basins_1zone','TLMO3','results_01','pars_optimal.csv'))
pars_2zone = pd.read_csv(os.path.join(folder,'basins_2zone','TLMO3','results_01','pars_optimal.csv'))

forcings_1zone = [pd.read_csv(os.path.join(folder,'basins_1zone','TLMO3','forcing_TLMO3-1.csv'))]
forcings_2zone = [pd.read_csv(os.path.join(folder,'basins_2zone','TLMO3','forcing_TLMO3-1.csv')),
                  pd.read_csv(os.path.join(folder,'basins_2zone','TLMO3','forcing_TLMO3-2.csv'))]

flow = pd.read_csv(os.path.join(folder,'basins_2zone','TLMO3','results_01','optimal_6h.csv'))
states = pd.read_csv(os.path.join(folder,'basins_2zone','TLMO3','results_01','optimal_states_6h.csv'))

basin = 'TLMO3'

fa_limits_basin = pd.read_csv(os.path.join(folder,'forcing-adj-limits_climo.csv'))
fa_limits_basin = fa_limits_basin[(fa_limits_basin['basin']==basin) &
                      (fa_limits_basin['zone']==basin+'-1')].copy()

In [114]:
flow.rename(columns={'sim_flow_cfs':'sim_flow_cfs_r'},inplace=True)

forcings_1zone[0].index = pd.to_datetime(forcings_1zone[0][['year', 'month', 'day', 'hour']])
for f in forcings_2zone:
    f.index = pd.to_datetime(f[['year', 'month', 'day', 'hour']])
flow.index = pd.to_datetime(flow[['year', 'month', 'day', 'hour']])
flow['datetime'] = flow.index

In [96]:
fa={}

for fa_par in pars.loc[pars['type']=='fa','name']:
    fa[fa_par]=pars[pars['name'] == fa_par]['value']

In [97]:
fa['map_scale']

32    1.135745
Name: value, dtype: float64

In [103]:
pars=pars_1zone
# fa adjustemnt parameters:  scale, p_redist, std, shift
map_fa_pars = pd.concat([fa['map_scale'], fa['map_p_redist'], fa['map_std'], fa['map_shift']]).to_numpy()
mat_fa_pars = pd.concat([fa['mat_scale'], fa['mat_p_redist'], fa['mat_std'], fa['mat_shift']]).to_numpy()
ptps_fa_pars = pd.concat([fa['ptps_scale'], fa['ptps_p_redist'], fa['ptps_std'], fa['ptps_shift']]).to_numpy()
pet_fa_pars = pd.concat([fa['pet_scale'], fa['pet_p_redist'], fa['pet_std'], fa['pet_shift']]).to_numpy()

In [102]:
map_fa_pars

array([1.1357447 , 0.16698601, 1.0240923 , 1.766347  ])

In [ ]:
print(s.sacsnow.__doc__)

In [105]:
m_1zone.map_fa_pars

array([1.1357447 , 0.16698601, 1.0240923 , 1.766347  ])

In [163]:
m_1zone = Model(forcings_1zone,pars_1zone,fa_limits_basin,flow)
sim_flow_cfs_1zone = m_1zone.run()

In [162]:
m_2zone = Model(forcings_2zone,pars_2zone,fa_limits_basin,flow)
sim_flow_cfs_2zone = m_2zone.run()

In [153]:
p['init_uztwc'].values

array([156.94437 ,  14.958142])

In [156]:
np.concatenate(([p['init_swe']],[p['init_uztwc']]),axis=0)
#np.concatenate(([[1,1]],[[2,2]]),axis=0)

array([[  0.      ,   0.      ],
       [156.94437 ,  14.958142]])

In [157]:
np.concatenate(([p['init_swe']], [p['init_uztwc']], [p['init_uzfwc']], [p['init_lztwc']],
        [p['init_lzfsc']], [p['init_lzfpc']], [p['init_adimc']]),axis=0)

array([[0.0000000e+00, 0.0000000e+00],
       [1.5694437e+02, 1.4958142e+01],
       [1.6720241e+01, 3.2377623e+02],
       [4.5087196e+02, 7.1507445e+02],
       [1.4001136e-02, 9.3521082e+01],
       [4.6718036e+00, 1.3037867e+02],
       [4.3506666e+02, 7.0641920e+02]])

In [116]:
pars_2zone

p = {}
for par in pars_2zone['name'].unique():
     p[par] =  pars_2zone[ pars_2zone['name'] == par]['value']

init = pd.concat([p['init_swe'], p['init_uztwc'], p['init_uzfwc'], p['init_lztwc'],
        p['init_lzfsc'], p['init_lzfpc'], p['init_adimc']]).to_numpy()

In [ ]:
#RUN LAGK#
ts_in=6
ts_out=6
units='ENGL'#'METR'
lagq=np.array([6, 0, 4, 10000, 3.5, 20000, 3, 40000, 3, 100000],dtype='f4')
len_lagq=len(lagq)/2
kq=np.array([1, 100, 1, 40000, 3, 100000],dtype='f4')
len_kq=len(kq)/2
init_co=float(6025)
init_if=float(6018)
init_of=float(6018)
init_stor=float(6307)

In [ ]:
p, c = s.pin7(ts_in,ts_out,len_lagq,len_kq,units,lagq,kq,init_co,init_if,init_of,init_stor)

In [ ]:
k_start=int(p[17])
k_len=int(p[k_start-1])
pina7_len=int(p[k_start+2*k_len])
p_end=k_start+2*(k_len+pina7_len)+1
p=p[:p_end]
c=c[:int(c[0])]

In [ ]:
c_copy=c.copy()
qb=s.flag7(p,c_copy,sakw1*0.0283168)
qc=s.fka7(p,c_copy,qb)

fortran_output=pd.DataFrame({'Lagged':qb,'Attenuated':qc},index=sakw1_chps.index)

#ts_test['Lagged']=qb*35.3147#qb[:len(ts)]*35.3147
#ts_test['Attenuated']=qc*35.3147#qb[:len(ts)]*35.3147
#ts_test.to_csv(os.path.join(folder,'Lag_Operation_Output.csv'))

d_start='2/5/1991'
d_end='2/15/1991'
    
plt.plot(sakw1_chps.loc[d_start:d_end].SAKW1*0.0283168,label='IN')
#plt.plot(fortran_output.loc[d_start:d_end].Lagged,label='Lagged')
plt.plot(fortran_output.loc[d_start:d_end].Attenuated,label='Attenuated')
#plt.plot(sakw1_chps.loc[d_start:d_end].SAKW1R*0.0283168,label='CHPS_Output')
#plt.plot(ts_test_lagged*0.0283168,label='Lagged')
#plt.plot(ts_test.index,qc[:len(ts)],label='Attenuated')

plt.xlabel('Timestep') 
plt.ylabel('Streamflow (cms)')

plt.legend()
 
plt.rcParams["figure.figsize"] = (10,10)
#ts_test['Lagged']=qb[:len(ts)]*35.3147
#ts_test['Attenuated']=qc[:len(ts)]*35.3147
#ts_test.to_csv(os.path.join(folder,'Lag_Operation_Output.csv'))

In [ ]:
fortran_output

In [ ]:
qb

In [ ]:
plt.plot(conw1_chps.iloc[:100].CONW1R*0.0283168,label='CHPS')
plt.plot(conw1_chps.iloc[:100].CONW1.index,qc,label='Attenuated')

